<p>
<font size='5' face='Georgia, Arial'>IIC-2233 Apunte Programación Avanzada</font><br>
<font size='1'> Ejercicios creados a partir de 2019-2 por Equipo Docente IIC2233. </font>
<font size='1'> Actualizados el 2023-2.</font>
</p>


# Ejercicios propuestos: Excepciones
## Excepciones Personalizadas

Los siguientes problemas se proveen como oportunidad de ejercitar los conceptos revisados en el material de **Excepciones**. Si tienes dudas sobre algún problema o alguna solución, no dudes en dejar una *issue* en el [foro del curso](https://github.com/IIC2233/syllabus/issues).

### Ejercicio 1: DCCopiones

Los estudiantes del curso Avanzación Programada acaban de entregar sus tareas. Sin embargo, existen sospechas de que algunos se copiaron entre ellos. Como en el DDC nos tomamos en serio el tema de la copia (espero que hayan leído el [código de honor](https://www.ing.puc.cl/wp-content/uploads/2015/08/cdigo-de-honor.pdf)), debemos penalizar a los estudiantes que incurran en esas prácticas. Es por eso que el profesor Ristián Cruz y el cuerpo de ayudantes de Avanzación Programada ¡te han delegado la tarea de completar el sistema para detectar casos de copia! Para que se haga más simple tu tarea, los códigos de los alumnos fueron pasados por una función de _hashing_ [SHA-256](https://en.wikipedia.org/wiki/SHA-2), es decir, todo el código fue transformado a un solo _string_ alfanumérico llamado _hash_, y están guardados dentro de la carpeta `estudiantes`. Tu misión se resume en revisar si existen estudiantes que tengan el mismo _hash_.

La clase `Corrector` es la que se encarga de revisar cada estudiantes, y levanta una excepción `ErrorCopia` cada vez que identifica un caso de copia. Debes crear la clase `ErrorCopia` (que hereda de `Exception` y modela una excepción). Cada instancia de `ErrorCopia` debe contener: el estudiante que fue pillado copiando (`estudiante`), el estudiante al que le copió (`copion`), el _hash_ que ambos comparten (`hash_copiado`). Además, debe llevar cuenta de cuantos casos de copia han ocurrido hasta el momento (`casos_de_copia`), y debe tener una estructura de datos que contenga a los estudiantes involucrados (`estudiantes_copiones`), en la cual no se repitan los estudiantes, si es que más de dos se copiaron entre sí.


In [ ]:
import os


# Retorna una lista con los estudiantes del curso.
def get_estudiantes(path):
    estudiantes = []
    with open(path, "r", encoding="utf-8") as archivo:
        for line in archivo.readlines():
            estudiantes.append(line.strip().split(",")[0])
    return estudiantes


class ErrorCopia(Exception):

    # Definir atributos de clase podría ser util...

    def __init__(self, estudiante, copion, hash_copiado):
        # COMPLETAR
        pass


class Corrector:

    def __init__(self):
        self.codigos_comprimidos = self.extract_codigos()
        self.estudiantes_revisados = dict()

    # Extrae los pares alumnos, código desde codigos_comprimidos.txt.
    def extract_codigos(self):
        codigos_comprimidos = dict()
        path_archivo = os.path.join("estudiantes", "codigos_comprimidos.txt")
        with open(path_archivo, "r", encoding="utf-8") as archivo:
            for linea in archivo.readlines():
                estudiante, hash_estudiante = linea.strip().split(",")
                codigos_comprimidos[estudiante] = hash_estudiante
        return codigos_comprimidos

    # Revisa si estudiante copió.
    def revisar(self, estudiante):
        hash_estudiante = self.codigos_comprimidos[estudiante]
        for estudiante_revisado in self.estudiantes_revisados:
            # Se detecta copia.
            if self.estudiantes_revisados[estudiante_revisado] == hash_estudiante:
                self.estudiantes_revisados[estudiante] = hash_estudiante
                # Aquí se levanta la excepción
                raise ErrorCopia(estudiante_revisado, estudiante, hash_estudiante)
        self.estudiantes_revisados[estudiante] = hash_estudiante


if __name__ == "__main__":
    lista_estudiantes = get_estudiantes(os.path.join("estudiantes", "codigos_comprimidos.txt"))

    corrector = Corrector()

    for nombre in lista_estudiantes:
        try:
            corrector.revisar(nombre)
        except ErrorCopia as error:
            print(f"Caso de copia encontrado entre {error.estudiante} y {error.copion}!")
            print(f"Ambos tienen el hash {error.hash_copiado}")
            print(f"Ya van {error.casos_de_copia} casos de copia :(")
            print()

    if ErrorCopia.estudiantes_copiones:
        print("Estudiantes que fueron pillados copiando:")
        for estudiante in ErrorCopia.estudiantes_copiones:
            print("\t - " + estudiante)
        print("Que pena :'(")
    else:
        print("No hubo casos de copia! Felicitaciones por la honestidad :)")

### Ejercicio 2: DCCompletos

El DDC ha decidido hacer una completada, porque nos gusta mucho comer. Pero se dieron cuenta de que es necesario tener un sistema para evitar que alguien le robe completos a otros y alguien se quede sin comer. Por eso inventaron un sistema de *tickets* y es necesario hacer la excepción personalizada `ErrorTratanDeRobar`, para levantarla cuando se encuentran duplicados y llevar un registro de cuantos duplicados se han encontrado. 

Se te entrega el esqueleto de la excepción `ErrorTratanDeRobar`. Esta es levantada por la clase `Tienda` cuando encuentra clientes con *tickets* duplicados. Cada instancia del error debe contener la siguiente información del error para que funcione el código descrito: el nombre del cliente como un atributo, el número del ticket como un atributo, y una *property* que entrega la cantidad de *tickets* duplicados que se han encontrado hasta ahora para ese *ticket*. Debes completar la excepción `ErrorTratanDeRobar` para que se logre el comportamiento esperado.

In [ ]:
from collections import namedtuple, defaultdict

# Modelamos los clientes.
Cliente = namedtuple('Cliente', ['ticket', 'nombre', 'apellido'])


class ErrorTratanDeRobar(Exception):

    # ¿Será buena idea un atributo de clase?

    def __init__(self, cliente):
        pass

    @property
    def cantidad_repeticiones(self):
        pass


# Modelamos la tienda con una estructura simple para almacenar sus clientes.
class Tienda:

    def __init__(self, nombre):
        self.nombre = nombre
        self.clientes = dict()

    # Método encargado de entregar un completo,
    # en caso de encontrar duplicado, lanza error
    def recibir(self, cliente):
        if cliente.ticket in self.clientes:
            raise ErrorTratanDeRobar(cliente)
        else:
            self.clientes[cliente.ticket] = cliente
            print(f'✅ Cliente {cliente.nombre} de ticket {cliente.ticket} '
                  'ha recibido su completo exitosamente.\n')


if __name__ == "__main__":

    clientes_que_no_han_comido = [
        Cliente('1', 'Daniela', 'Poblete'),
        Cliente('2', 'Tomás', 'González'),
        Cliente('3', 'Enzo', 'Tamburirni'),
        Cliente('4', 'Daniela', 'Concha'),
        Cliente('2', 'Dante', 'Pinto'),
        Cliente('5', 'Juan', 'Aguillon'),
        Cliente('1', 'Caua', 'Paz'),
        Cliente('6', 'Máx', 'Narea'),
        Cliente('1', 'Javiera', 'Ochoa')
    ]

    DCCompletos = Tienda('DDCompletos')

    for cliente in clientes_que_no_han_comido:
        try:
            DCCompletos.recibir(cliente)
        except ErrorTratanDeRobar as error:
            print(f'❌ Cliente {error.nombre} tratando de robar un completo '
                  f'con el ticket {error.ticket} encontrado')
            print(f'Ya van {error.cantidad_repeticiones} repetidos para el '
                  f'ticket {error.ticket}')
            print()